In [1]:
import pandas as pd
import numpy as np
from taigapy import create_taiga_client_v3
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
Model = pd.read_pickle("pydata/model_data.pkl")

In [3]:
Model

,ModelID,PatientID,CellLineName,StrippedCellLineName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,PatientSubtypeFeatures,...,EngineeredModelDetails,CulturedResistanceDrug,PublicComments,CCLEName,HCMIID,ModelAvailableInDbgap,ModelSubtypeFeatures,WTSIMasterCellID,SangerModelID,COSMICID
0,ACH-000001,PT-gj46wT,NIH:OVCAR-3,NIHOVCAR3,HGSOC,Ovary/Fallopian Tube,Ovarian Epithelial Tumor,High-Grade Serous Ovarian Cancer,HGSOC,None,...,None,None,None,NIHOVCAR3_OVARY,None,Available in SRA,None,2201.0,SIDM00105,905933.0
1,ACH-000002,PT-5qa3uk,HL-60,HL60,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"TP53(del), CDKN2A and NRAS mutations [PubMed=2...",...,None,None,None,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,55.0,SIDM00829,905938.0
2,ACH-000003,PT-puKIyc,CACO2,CACO2,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,...,None,None,None,CACO2_LARGE_INTESTINE,None,Available in SRA,None,NaN,SIDM00891,NaN
3,ACH-000004,PT-q4K2cp,HEL,HEL,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"JAK2 and TP53 mutations,",...,None,None,None,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,783.0,SIDM00594,907053.0
4,ACH-000005,PT-q4K2cp,HEL 92.1.7,HEL9217,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,JAK2 and TP53 mutations,...,None,None,None,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,None,Available in SRA,None,NaN,SIDM00593,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2547,ACH-003659,PT-UFmFPI,SMS-KAN,SMSKAN,NBL,Peripheral Nervous System,Neuroblastoma,Neuroblastoma,NBL,None,...,None,None,None,None,None,None,None,NaN,None,NaN
2548,ACH-003660,PT-UFmFPI,SMS-KANR,SMSKANR,NBL,Peripheral Nervous System,Neuroblastoma,Neuroblastoma,NBL,None,...,None,None,None,None,None,None,None,NaN,None,NaN
2549,ACH-003665,PT-ZwaNYv,MBU-8,MBU8,AML,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,AML,"Compared to the parent cell line, has a comple...",...,None,None,None,None,None,None,None,NaN,None,NaN
2550,ACH-003666,PT-ijqCP4,YCUB-4,YCUB4,BLL,Lymphoid,B-Lymphoblastic Leukemia/Lymphoma,B-Lymphoblastic Leukemia/Lymphoma,BLL,cells express a MEF2D exon5/BCL9 exon 9 fusion,...,None,None,None,None,None,None,MEF2D,NaN,None,NaN


In [4]:
Model.set_index('ModelID', inplace=True)

In [5]:
E3_expr_filtered = pd.read_pickle("pydata/e3_expression_filtered.pkl")

In [6]:
crispr_dependencies_3_filtered = pd.read_pickle("pydata/tfs_dependencies_filtered.pkl")

In [7]:
crispr_dependencies_3_filtered

,TFAP2A,TFAP2B,TFAP2C,TFAP2D,TFAP2E,ARID3A,ARID3B,ARID3C,ARID5A,ARID5B,...,THAP6,THAP7,THAP8,THAP9,BMAL1,BMAL2,ZNF875,CBLL2,ZUP1,TBXT
ACH-000001,0.032275,-0.052657,-0.099978,-0.040024,0.060890,0.102625,-0.121782,0.027268,-0.243864,-0.096952,...,-0.087478,-0.317241,-0.052855,-0.065466,-0.645930,0.140806,-0.892911,0.215270,-0.014496,0.169385
ACH-000004,0.145169,0.066916,-0.074218,-0.125423,0.240744,-0.105335,-0.734358,-0.027738,-0.136107,0.008301,...,-0.462663,-0.194145,0.130992,-0.251477,-0.205806,-0.090015,0.168027,0.280951,-0.182743,0.082328
ACH-000005,0.171513,-0.361703,-0.017947,-0.103094,0.315080,0.069369,-0.026263,-0.022071,0.006219,0.200501,...,-0.094047,-0.180209,-0.163869,-0.026951,-0.207084,-0.134599,0.374316,-0.018200,-0.283811,0.058674
ACH-000007,-0.073170,0.200249,-0.292673,-0.238197,-0.059635,0.041934,-0.329979,-0.080568,-0.067746,-0.198359,...,-0.071877,-0.105574,0.144103,-0.023396,0.140064,-0.069550,0.180594,0.104287,-0.311871,-0.006005
ACH-000009,0.224490,-0.001485,-0.172446,0.018604,0.034670,-0.316335,-0.053869,0.059077,-0.058235,-0.495861,...,-0.059427,-0.024232,-0.025614,-0.115766,-0.111904,0.027901,0.098494,0.060150,0.014514,0.136429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-003578,-0.070363,-0.033106,-0.243238,-0.370448,-0.108596,0.023995,-0.181551,-0.213355,-0.284968,-0.052868,...,0.040754,-0.215718,0.054442,-0.043687,-0.052162,-0.106350,0.018423,0.079413,0.005381,0.105370
ACH-003580,-0.081792,-0.181257,-0.381345,-0.202155,0.109083,-0.004828,0.018773,-0.112690,-0.071700,-0.386106,...,-0.040535,-0.178553,0.195923,0.063442,-0.119621,-0.077457,-0.205756,-0.021425,0.016575,0.077086
ACH-003587,-0.128495,-0.347200,-0.229025,-0.420290,0.106401,-0.087998,0.062317,-0.158840,0.009397,-0.086930,...,0.022615,0.289244,0.065706,0.043732,-0.095881,0.210714,-0.104913,-0.141233,0.058059,0.013758
ACH-003590,0.029615,-0.090314,-0.087541,-0.343993,0.073768,-0.087375,-0.049801,-0.047360,0.009253,-0.328680,...,0.038728,-0.037614,0.067970,0.051580,-0.043185,-0.007903,-0.250206,-0.080775,0.102683,0.069626


In [8]:
Model_filtered = Model[Model.index.isin(E3_expr_filtered.index)]

In [9]:
for lineage in Model_filtered['OncotreeLineage'].unique():
    print (lineage)

Ovary/Fallopian Tube
Myeloid
Bowel
Bladder/Urinary Tract
Lung
Skin
Breast
Pancreas
Lymphoid
CNS/Brain
Soft Tissue
Bone
Peripheral Nervous System
Pleura
Prostate
Kidney
Fibroblast
Biliary Tract
Esophagus/Stomach
Head and Neck
Thyroid
Ampulla of Vater
Uterus
Liver
Cervix
Eye
Vulva/Vagina
Adrenal Gland
Testis
None
Other


### Code to get correlation by lineage
It takes a while to run